### Function

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import combine_svm
import pandas as pd
from tqdm import tqdm
import csv
import scipy.signal
from scipy.fftpack import fft
import seaborn as sns
sns.set()
''' ----------------------- Movement ----------------------- '''
# 30s訊號 = 30 * 20 = 600
def mov_dens_fn(raw_sig):
    new_sig = []
    count = 0
    for num in range(80):
        top = 0
        first = int(num*(0.5*20))
        last = int((num+1)*(0.5*20))
        x = raw_sig[first:last]

        # 方差公式
        for i in range(10):
            top += np.square(x[i] - np.average(x))
        result = top / (10 - 1)
        if result > 0.005:  # 閥值可調
            count += 1
    percent = (count/80) * 100
    return percent

''' ----------------------- Respiration ----------------------- '''
# 10個fRSA
def tfRSA_fn(fRSA_sig):
    tfRSA = np.std(fRSA_sig)
    return tfRSA

# 31個f𝑅𝑆𝐴
def sfRSA_fn(fRSA_sig):
    sfRSA = scipy.signal.savgol_filter(fRSA_sig, 31, 3)
    sfRSA_mean = np.average(sfRSA)
    return sfRSA, sfRSA_mean

# 31個tf𝑅𝑆𝐴
def stfRSA_fn(tfRSA_sig):
    stfRSA = scipy.signal.savgol_filter(np.array(tfRSA_sig), 31, 2)
    stfRSA_mean = np.average(stfRSA)
    return stfRSA, stfRSA_mean

# 31個f𝑅𝑆𝐴
def sdfRSA_fn(fRSA, sfRSA):
    sdfRSA = np.abs(fRSA - sfRSA)
    sdfRSA = scipy.signal.savgol_filter(sdfRSA, 31, 3)
    sdfRSA_mean = np.average(sdfRSA)
    return sdfRSA, sdfRSA_mean

''' ----------------------- Heart rate ----------------------- '''
def tmHR_fn(mHR_sig):
    return tfRSA_fn(mHR_sig)

def smHR_fn(mHR_sig):
    return sfRSA_fn(mHR_sig)

def stmHR_fn(tmHR_sig):
    return stfRSA_fn(tmHR_sig)

def sdmHR_fn(mHR, smHR):
    return sdfRSA_fn(mHR, smHR)

# ---------------- 待補 ---------------- 
def LF_HF_LFHF(sig):
    LF_sig = combine_svm.iir_bandpass_filter_1(sig, 0.04, 0.15, 20, 9, "cheby2")
    HF_sig = combine_svm.iir_bandpass_filter_1(sig, 0.15, 0.4, 20, 9, "cheby2")
    print(LF_sig)
    LF_eng = energe(LF_sig)
    HF_eng = energe(HF_sig)
    LFHF_eng = HF_eng / LF_eng
    return LF_eng, HF_eng, LFHF_eng

def energe(sig):
    N = len(sig)
    bps_fft = np.fft.fft(sig)
    return np.sum(np.abs(bps_fft[:N // 2])**2)

def sHF_fn(HF_sig):
    sHF = scipy.signal.savgol_filter(HF_sig, 31, 3)
    sHF_mean = np.average(sfRSA)
    return sHF, sHF_mean

def sLFHF_fn(LFHF_sig):
    sLFHF = scipy.signal.savgol_filter(LFHF_sig, 31, 3)
    sLFHF_mean = np.average(sfRSA)
    return sLFHF, sLFHF_mean


### 讀所有資料

In [19]:
names = "./dataset_sleep_test"
raw_data_pd = pd.DataFrame()
for name in os.listdir(names):
    files = os.path.join(names, name, "0.8", "processed_data")
    print(files)
    for num in range(len(os.listdir(files))):
        datas = os.listdir(files)[num]
        data = os.path.join(files, datas)
        print(data)
        raw_data = pd.read_csv(data)
        print(raw_data.shape)
        raw_data_pd = raw_data_pd.append(raw_data, ignore_index=True)
raw_data_pd.info()
"""
names = "./dataset_sleep"
for name in os.listdir(names):
    files = os.path.join(names, name, "0.8")
    for num in range(len(os.listdir(files)) // 2):
        datas = os.listdir(files)[num]
        data = os.path.join(files, datas)
        raw_data = pd.read_csv(data)
        raw_data_pd = pd.DataFrame(raw_data)
raw_data_pd.info()
"""

./dataset_sleep_test\0.8\0.8\processed_data
./dataset_sleep_test\0.8\0.8\processed_data\0_processed_data.csv
(565183, 39)
./dataset_sleep_test\0.8\0.8\processed_data\1_processed_data.csv
(584380, 39)
./dataset_sleep_test\0.8\0.8\processed_data\2_processed_data.csv
(561584, 39)
./dataset_sleep_test\0.8\0.8\processed_data\3_processed_data.csv
(541183, 39)
./dataset_sleep_test\0.8\0.8\processed_data\4_processed_data.csv
(569984, 39)
./dataset_sleep_test\0.8\0.8\processed_data\5_processed_data.csv
(586782, 39)
./dataset_sleep_test\0.8\0.8\processed_data\6_processed_data.csv
(565184, 39)
./dataset_sleep_test\0.8\0.8\processed_data\7_processed_data.csv
(603583, 39)
./dataset_sleep_test\0.8\0.8\processed_data\8_processed_data.csv
(561583, 39)
./dataset_sleep_test\0.8\0.8\processed_data\9_processed_data.csv
(531585, 39)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5671031 entries, 0 to 5671030
Data columns (total 39 columns):
 #   Column                           Dtype  
---  ------      

'\nnames = "./dataset_sleep"\nfor name in os.listdir(names):\n    files = os.path.join(names, name, "0.8")\n    for num in range(len(os.listdir(files)) // 2):\n        datas = os.listdir(files)[num]\n        data = os.path.join(files, datas)\n        raw_data = pd.read_csv(data)\n        raw_data_pd = pd.DataFrame(raw_data)\nraw_data_pd.info()\n'

In [ ]:
# 特徵單位變成秒
raw_data_pd['heart'] = raw_data_pd['heart'].replace(0, np.nan)
new_data = raw_data_pd.dropna()
new_data.head(-10)

### 只取有心律跟呼吸律的時間點，把特徵單位轉成秒

In [41]:
# 特徵單位變成秒
raw_data_pd['heart'] = raw_data_pd['heart'].replace(0, np.nan)
new_data = raw_data_pd.dropna()
new_data.head(-10)
#new_data.info()

,rangeBinIndexMax,rangeBinIndexPhase,maxVal,processingCyclesOut,processingCyclesOut1,rangeBinStartIndex,rangeBinEndIndex,unwrapPhasePeak_mm,outputFilterBreathOut,outputFilterHeartOut,...,rsv[4],rsv[5],rsv[6],rsv[7],rsv[8],rsv[9],datetime,heart,breath,sleep
0,0,22,139510784,9247,0,11,73,52.404808,-0.323164,-0.621020,...,0,0,0,0,0,0,00:10:59,59.2603,18.5054,5
7,0,22,107942984,9245,0,11,73,55.756603,0.036978,0.420097,...,0,0,0,0,0,0,00:11:00,59.2090,18.4658,5
27,0,22,263363424,9246,0,11,73,56.356892,0.309814,1.666175,...,0,0,0,0,0,0,00:11:01,59.1797,18.3809,5
47,0,22,257653728,9247,0,11,73,63.770531,0.136261,2.184742,...,0,0,0,0,0,0,00:11:02,59.2676,18.2959,5
67,0,22,20798972,9246,0,11,73,69.206299,0.292031,-1.563346,...,0,0,0,0,0,0,00:11:03,59.2676,18.1406,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565749,0,31,249442640,9252,0,11,73,764.500916,-0.272279,-0.555051,...,0,0,0,0,0,0,08:02:28,77.1304,19.8369,5
565769,0,31,402681536,9258,0,11,73,764.453918,0.318991,0.977882,...,0,0,0,0,0,0,08:02:29,78.8288,19.8384,5
565789,0,31,372125888,9257,0,11,73,761.422913,0.074580,0.023633,...,0,0,0,0,0,0,08:02:30,76.2330,19.8442,5
565809,0,31,173766688,9259,0,11,73,755.635986,-0.070902,-0.110535,...,0,0,0,0,0,0,08:02:31,77.5918,19.7490,5


### 特徵: tfRSA, tmHR, sfRSA, smHR, sdfRSA, sdmHR

In [14]:
breath = np.array(new_data['breath'])
heart = np.array(new_data['heart'])
print(f"Total len: {len(heart)}\n")

# ------------- tfRSA and tmHR ------------- 
local_tfRSA = []
local_tmHR = []
num_in_window = 10

# 空值補0
for i in range(num_in_window-1):
    local_tfRSA.append(0)
    local_tmHR.append(0)

for turn in range(len(breath) - num_in_window + 1):
    start_index = turn
    end_index = start_index + num_in_window

    # Slide data
    window_breath = breath[start_index:end_index]
    window_heart = heart[start_index:end_index]

    # Standard deviation
    local_tfRSA.append(str(round(tfRSA_fn(window_breath), 4)))
    local_tmHR.append(str(round(tmHR_fn(window_heart), 4)))

print("tfRSA and tmHR Complete!")
print(f"Real len: {len(breath)}\ntfRSA len: {len(local_tfRSA)}\ntmHR len: {len(local_tmHR)}")

#  ------------- sfRSA and smHR and sdfRSA and sdmHR------------- 
local_sfRSA = []
local_sdfRSA = []
local_smHR = []
local_sdmHR = []
num_in_window = 31

# 空值補0
for i in range(num_in_window-1):
    local_sfRSA.append(0)
    local_smHR.append(0)
    local_sdfRSA.append(0)
    local_sdmHR.append(0)

for turn in range(len(breath) - num_in_window + 1):
    start_index = turn
    end_index = start_index + num_in_window

    # Slide data
    window_breath = breath[start_index:end_index]
    window_heart = heart[start_index:end_index]

    # Savitzky–Golay filter
    sfRSA, sfRSA_mean = sfRSA_fn(window_breath)
    sdfRSA, sdfRSA_mean = sdfRSA_fn(window_breath, sfRSA)
    smHR, smHR_mean = smHR_fn(window_heart)
    sdmHR, sdmHR_mean = sdmHR_fn(window_heart, smHR)
    local_sfRSA.append(round(sfRSA_mean, 4))
    local_sdfRSA.append(round(sdfRSA_mean, 4))
    local_smHR.append(round(smHR_mean, 4))
    local_sdmHR.append(round(sdmHR_mean, 4))

print("\nsfRSA and smHR Complete!")
print(f"Real len: {len(heart)}\nsfRSA len: {len(local_sfRSA)}\nsmHR len: {len(local_smHR)}")

# 插入特徵
new_data.insert(39, "tfRSA", local_tfRSA)
new_data.insert(40, "tmHR", local_tmHR)
new_data.insert(41, "sfRSA", local_sfRSA)
new_data.insert(42, "smHR", local_smHR)
new_data.insert(43, "sdfRSA", local_sdfRSA)
new_data.insert(44, "sdmHR", local_sdmHR)
new_data.head(10)

Total len: 28140

tfRSA and tmHR Complete!
Real len: 28140
tfRSA len: 28140
tmHR len: 28140

sfRSA and smHR Complete!
Real len: 28140
sfRSA len: 28140
smHR len: 28140


,rangeBinIndexMax,rangeBinIndexPhase,maxVal,processingCyclesOut,processingCyclesOut1,rangeBinStartIndex,rangeBinEndIndex,unwrapPhasePeak_mm,outputFilterBreathOut,outputFilterHeartOut,...,datetime,heart,breath,sleep,tfRSA,tmHR,sfRSA,smHR,sdfRSA,sdmHR
0,0,24,361640000,9250,0,11,73,12.411730,-0.288751,-0.197697,...,00:14:00,83.3111,16.8384,5,0,0,0.0,0.0,0.0,0.0
20,0,25,539577216,9249,0,11,73,9.486880,0.147601,0.017571,...,00:14:01,82.8943,16.8091,5,0,0,0.0,0.0,0.0,0.0
40,0,25,329165504,9252,0,11,73,11.948972,0.138334,0.047295,...,00:14:02,82.0496,16.7725,5,0,0,0.0,0.0,0.0,0.0
60,0,25,1022882944,9252,0,11,73,11.836657,0.002434,0.105520,...,00:14:03,82.1838,16.7842,5,0,0,0.0,0.0,0.0,0.0
80,0,25,276341888,9254,0,11,73,10.081087,-0.087562,0.092756,...,00:14:04,81.6076,16.7051,5,0,0,0.0,0.0,0.0,0.0
100,0,25,486458688,9253,0,11,73,5.921892,0.031427,-0.094766,...,00:14:05,81.0495,16.5996,5,0,0,0.0,0.0,0.0,0.0
120,0,25,753500416,9247,0,11,73,7.079786,0.022268,-0.029966,...,00:14:06,81.6754,16.5117,5,0,0,0.0,0.0,0.0,0.0
140,0,24,186557904,9248,0,11,73,13.776101,0.021727,-0.025970,...,00:14:07,80.0450,16.4238,5,0,0,0.0,0.0,0.0,0.0
160,0,24,811545792,9253,0,11,73,14.008210,-0.125044,-0.016622,...,00:14:08,78.6370,16.5088,5,0,0,0.0,0.0,0.0,0.0
180,0,24,325669792,9253,0,11,73,16.002903,0.064682,0.018934,...,00:14:09,80.3709,16.7036,5,0.1379,1.33,0.0,0.0,0.0,0.0


### 為了要算 stfRSA and stmHR 將特徵單位變成tmHR

In [15]:
# 特徵單位變成tmHR
new_data['tmHR'] = new_data['tmHR'].replace(0, np.nan)
new_data = new_data.dropna()
new_data

C:\Users\lian\anaconda3\envs\env_radar\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,rangeBinIndexMax,rangeBinIndexPhase,maxVal,processingCyclesOut,processingCyclesOut1,rangeBinStartIndex,rangeBinEndIndex,unwrapPhasePeak_mm,outputFilterBreathOut,outputFilterHeartOut,...,datetime,heart,breath,sleep,tfRSA,tmHR,sfRSA,smHR,sdfRSA,sdmHR
180,0,24,325669792,9253,0,11,73,16.002903,0.064682,0.018934,...,00:14:09,80.3709,16.7036,5,0.1379,1.33,0.0000,0.0000,0.0000,0.0000
200,0,24,169395152,9249,0,11,73,12.219460,-0.188703,-0.236100,...,00:14:10,80.4267,17.0171,5,0.1675,1.1854,0.0000,0.0000,0.0000,0.0000
220,0,24,1042879872,9244,0,11,73,12.526545,0.108156,0.078113,...,00:14:11,80.6919,17.1021,5,0.2077,1.024,0.0000,0.0000,0.0000,0.0000
240,0,24,113452352,9250,0,11,73,13.370313,0.189698,-0.181984,...,00:14:12,80.9171,17.1855,5,0.2518,0.9475,0.0000,0.0000,0.0000,0.0000
260,0,24,28351908,9249,0,11,73,16.282375,0.006350,0.353688,...,00:14:13,80.0534,17.2412,5,0.2915,0.8365,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562683,0,31,406061792,9241,0,11,73,159.862732,0.015230,-0.053527,...,08:02:55,78.8911,15.0645,5,0.5116,2.395,18.3203,64.9868,0.0693,3.9928
562703,0,30,302971936,9238,0,11,73,164.002441,0.024880,0.002923,...,08:02:56,79.4479,14.8682,5,0.5194,2.2375,18.1049,65.9201,0.0695,3.8494
562724,0,30,123029968,9251,0,11,73,163.419373,-0.197981,0.398763,...,08:02:57,79.7688,14.6895,5,0.5369,2.0783,17.8894,66.7293,0.0696,3.5310
562744,0,30,254270944,9251,0,11,73,162.185303,0.035512,-0.065868,...,08:02:58,79.4805,14.4155,5,0.5605,1.8701,17.6707,67.5298,0.0728,2.8886


### 特徵: stfRSA and stmHR

In [16]:
tfRSA = np.array(new_data['tfRSA'])
tmHR = np.array(new_data['tmHR'])

#  ------------- stfRSA and stmHR ------------- 
local_stfRSA = []
local_stmHR = []
num_in_window = 31

# 空值補0
for i in range(num_in_window-1):
    local_stfRSA.append(0)
    local_stmHR.append(0)

for turn in range(len(tfRSA) - num_in_window + 1):
    start_index = turn
    end_index = start_index + num_in_window

    # Slide data
    window_tfRSA = tfRSA[start_index:end_index]
    window_tmHR = tmHR[start_index:end_index]

    # Savitzky–Golay filter
    stfRSA, stfRSA_mean = stfRSA_fn(window_tfRSA)
    stmHR, stmHR_mean = stmHR_fn(window_tmHR)
    local_stfRSA.append(round(stfRSA_mean, 4))
    local_stmHR.append(round(stmHR_mean, 4))

new_data.insert(45, "stfRSA", local_stfRSA)
new_data.insert(46, "stmHR", local_stmHR)

print("\nsfRSA and smHR Complete!")
print(f"Real len: {len(heart)}\nstfRSA len: {len(local_stfRSA)}\nstmHR len: {len(local_stmHR)}")


sfRSA and smHR Complete!
Real len: 28140
stfRSA len: 28131
stmHR len: 28131


### 只保留不為零的特徵

In [17]:
# 只保留不為零的特徵
new_data['stmHR'] = new_data['stmHR'].replace(0, np.nan)
new_data = new_data.dropna()
new_data

,rangeBinIndexMax,rangeBinIndexPhase,maxVal,processingCyclesOut,processingCyclesOut1,rangeBinStartIndex,rangeBinEndIndex,unwrapPhasePeak_mm,outputFilterBreathOut,outputFilterHeartOut,...,breath,sleep,tfRSA,tmHR,sfRSA,smHR,sdfRSA,sdmHR,stfRSA,stmHR
780,0,25,306160256,9253,0,11,73,17.834946,0.097114,-0.500412,...,18.0205,5,0.5829,1.1531,17.0225,79.3458,0.0943,0.6883,0.2663,0.9823
800,0,25,613976640,9246,0,11,73,17.494511,-0.147979,-0.191818,...,18.1245,5,0.5687,1.45,17.0684,79.1446,0.1122,0.7032,0.2802,0.9861
820,0,25,337541888,9254,0,11,73,20.005903,0.004617,-0.082740,...,18.0952,5,0.5279,1.456,17.1031,79.0549,0.1500,0.8249,0.2918,0.9949
840,0,25,379938752,9251,0,11,73,18.024300,-0.133945,0.043885,...,18.2271,5,0.4852,1.5784,17.1394,79.0037,0.1815,0.8831,0.3008,1.0128
860,0,25,491102016,9253,0,11,73,17.030777,-0.038763,0.149916,...,18.1978,5,0.4216,1.6305,17.1721,78.9475,0.1992,0.8536,0.3063,1.0348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562683,0,31,406061792,9241,0,11,73,159.862732,0.015230,-0.053527,...,15.0645,5,0.5116,2.395,18.3203,64.9868,0.0693,3.9928,0.5930,6.0131
562703,0,30,302971936,9238,0,11,73,164.002441,0.024880,0.002923,...,14.8682,5,0.5194,2.2375,18.1049,65.9201,0.0695,3.8494,0.6023,5.8625
562724,0,30,123029968,9251,0,11,73,163.419373,-0.197981,0.398763,...,14.6895,5,0.5369,2.0783,17.8894,66.7293,0.0696,3.5310,0.6097,5.7070
562744,0,30,254270944,9251,0,11,73,162.185303,0.035512,-0.065868,...,14.4155,5,0.5605,1.8701,17.6707,67.5298,0.0728,2.8886,0.6156,5.5538


In [18]:
new_data.to_csv("./sleep_features/test.csv", index=False)
print("Completed!")

Completed!


# Extraction Feature

In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [127]:
""" 讀入資料 """
data_path = "./sleep_features/train.csv"
total_data = pd.read_csv(data_path)
x = pd.DataFrame(total_data[["heart", "breath", "tfRSA", "tmHR", "sfRSA", "smHR", "sdfRSA", "sdmHR", "stfRSA", "stmHR"]])
y = pd.DataFrame(total_data[["sleep"]])

In [10]:
""" 讀入資料 """
from sklearn import preprocessing

import joblib
data_path = "./sleep_features/test.csv"
total_data = pd.read_csv(data_path)
x = pd.DataFrame(total_data[["heart", "breath", "tfRSA", "tmHR", "sfRSA", "smHR", "sdfRSA", "sdmHR", "stfRSA", "stmHR"]])
y = pd.DataFrame(total_data[["sleep"]])
rf2 = joblib.load('save/sleep_feature.pkl')
x = np.array(x)
from sklearn.metrics import accuracy_score
#X = preprocessing.scale(x)

print(accuracy_score(rf2.predict(x), y))
print(rf2.predict(x))


0.4823998787940003
[3 3 3 ... 4 4 4]


In [128]:
randomForestModel = RandomForestClassifier(n_estimators = 100, random_state = 69, n_jobs = -1, min_samples_leaf = 3, min_samples_split = 5)
randomForestModel.fit(x, y)
import joblib
joblib.dump(randomForestModel, "save/sleep_feature.pkl")


C:\Users\lian\anaconda3\envs\env_radar\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


['save/sleep_feature.pkl']

In [122]:
import joblib
joblib.dump(randomForestModel, "save/sleep_feature.pkl")

['save/sleep_feature.pkl']

In [123]:
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(n_estimators = 100, random_state = 69, n_jobs = -1, min_samples_leaf = 3, min_samples_split = 5)
scores = cross_val_score(rf, x, y, cv=2, scoring='accuracy')
print(scores)
print(scores.mean())

C:\Users\lian\anaconda3\envs\env_radar\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\lian\anaconda3\envs\env_radar\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[0.53109397 0.54283083]
0.5369623993231694


In [124]:
plot_train = pd.DataFrame(X_train)
plot_train["class"] = y_train
plot_test = pd.DataFrame(X_test)
plot_test["class"] = y_test

In [125]:
ac = 0
for i in range(len(predict)):
    if predict[i] == gt[i]:
        ac +=1

print(ac / len(gt))

0.9624102544825771
